In [1]:
%matplotlib inline
import os
import re
import io
import itertools
import pprint

from bokeh.io import output_notebook, show
from bokeh.layouts import gridplot
from bokeh.plotting import figure
from bokeh.models import Range1d, Legend
from bokeh.palettes import all_palettes

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import xarray as xr

import climatools.lblnew.bestfit_params as bestfits
from climatools.lblnew import setup_bestfit, setup_overlap
import climatools.lblnew.pipeline as pipe_lblnew
import climatools.cliradlw.setup as setup_cliradlw
import climatools.cliradlw.pipeline as pipe_cliradlw

import climatools.html.html as climahtml
from climatools.lblnew.dataio import *
from climatools.plot.plot import *


import IPython.display as display

In [2]:
%run param.py

In [3]:
def load_output_file(path_csv):
    '''
    Load lblnew output .csv file to xarray.Dataset
    
    Parameters
    ----------
    path_csv: str
              Path to the .csv file to be loaded.
    ds: xarray.Dataset
        Data in the input file in the form of an xarray.Dataset.
    '''
    toindex = ['band', 'pressure', 'igg', 'g']    
    df = pd.read_csv(path_csv, sep=r'\s+')
    df = df.set_index([i for i in toindex if i in df.columns])
    df = df.rename(columns={'sfu': 'flug',
                            'sfd': 'fldg',
                            'fnet': 'fnetg',
                            'coolr': 'coolrg'})
    ds = xr.Dataset.from_dataframe(df)

    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    return ds

In [4]:
def lblnew_setup(param=None):
    if 'ng_refs' in param:
        return {'setup': setup_bestfit,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wfluxg.dat',
                'fname_cool_wgt': 'output_wcoolrg.dat'}
    else:
        return {'setup': setup_overlap,
                'fname_flux_crd': 'output_flux.dat',
                'fname_cool_crd': 'output_coolr.dat',
                'fname_flux_wgt': 'output_wflux.dat',
                'fname_cool_wgt': 'output_wcoolr.dat'}

    
def load_lblnew_data(param):
    
    fname_dsname = [('fname_flux_crd', 'ds_flux_crd'),
                    ('fname_cool_crd', 'ds_cool_crd'),
                    ('fname_flux_wgt', 'ds_flux_wgt'),
                    ('fname_cool_wgt', 'ds_cool_wgt')]
    
    d = lblnew_setup(param)
    dir_fortran = pipe_lblnew.get_dir_case(param, setup=d['setup'])
    
    data_dict = {}
    for fname, dsname in fname_dsname:
        fpath = os.path.join(dir_fortran, d[fname])
        data_dict[dsname] = load_output_file(fpath)
    return data_dict
    

    
    

In [5]:
d = load_lblnew_data(PARAM_LBLNEW)

DS_FLUX_CRD = d['ds_flux_crd']
DS_COOL_CRD = d['ds_cool_crd']
DS_FLUX_WGT = d['ds_flux_wgt']
DS_COOL_WGT = d['ds_cool_wgt']

In [6]:
DIR_FORTRAN = pipe_cliradlw.get_fortran_dir(PARAM, 
                                            setup=setup_cliradlw)

PATH_FLUX = os.path.join(DIR_FORTRAN, 'output_flux.dat')
PATH_COOL = os.path.join(DIR_FORTRAN, 'output_coolr.dat')

DS_FLUX = load_output_file(PATH_FLUX)
DS_COOL = load_output_file(PATH_COOL)

In [7]:
output_notebook()

Loading BokehJS ...

In [8]:
def fmt_cool(ds_in):
    ds = ds_in.copy(deep=True)
    if 'igg' in ds.dims:
        ds = ds.sel(igg=1)

    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band')
        except ValueError:
            ds = ds.sum('band')
                
    return ds['coolrg']


def pltdata_cooling_1(ds_cool=None,
                      ds_cool_crd=None, ds_cool_wgt=None):
    


    colors = all_palettes['Set1'][4]
    data = [{'label': 'CLIRAD `coolr_bands`',
             'srs': fmt_cool(ds_cool),
             'line_dash': 'dashed', 'line_width': 5,
             'color': colors[0], 'alpha': .6},
            {'label': 'CRD',
             'srs': fmt_cool(ds_cool_crd),
             'line_dash': 'solid', 'line_width': 1.5,
             'marker': 'circle', 'marker_size': 5,
             'color': colors[2], 'alpha': 1,},
            {'label': 'WGT igg=10 wgt_flux=2',
             'srs': fmt_cool(ds_cool_wgt),
             'line_dash': 'solid', 'line_width': 3,
             'marker': 'square', 'marker_size': 5,
             'color': colors[3], 'alpha': .6}]
    return data


def nice_xlims(pltdata=None, prange=None):
    
    def get_slice(srs):
        return srs.sel(pressure=slice(*prange))
    
    srss = [d['srs'] for d in pltdata]
    vmin = min([get_slice(srs).min() for srs in srss])
    vmax = max([get_slice(srs).max() for srs in srss])
    dv = (vmax - vmin) * .01
    return float(vmin - dv), float(vmax + dv)
    

def plt_cooling_bokeh(pltdata=None):
    '''
    Plot a list of cooling rate profiles using bokeh.
    '''
    ymin = 1e-2 
    ymax = 1020 
        
    p1 = figure(title="Linear pressure scale", 
                plot_width=400)        
    xmin, xmax = nice_xlims(pltdata, prange=(50, 1050))
    for d in pltdata:
        if 'marker' in d:
            getattr(p1, d['marker'])(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=.7)
        p1.line(d['srs'].values, 
                d['srs'].coords['pressure'].values,
                color=d['color'], alpha=d['alpha'], 
                line_width=d['line_width'], line_dash=d['line_dash'])
    
    p1.y_range = Range1d(ymax, ymin)
    p1.yaxis.axis_label = 'pressure [mb]'   
    p1.x_range = Range1d(xmin, xmax)
    p1.xaxis.axis_label = 'cooling rate [K/day]'
            
    p2 = figure(title='Log pressure scale', y_axis_type='log',
                plot_width=560)
    xmin, xmax = nice_xlims(pltdata, prange=(.01, 200))
    
    rs = []
    for d in pltdata:
        rd = []
        if 'marker' in d:
            r_mark = getattr(p2, d['marker'])(d['srs'].values, 
                        d['srs'].coords['pressure'].values,
                        color=d['color'], alpha=.7)
            rd.append(r_mark)
        r_line = p2.line(d['srs'].values, 
                    d['srs'].coords['pressure'].values,
                    color=d['color'], alpha=d['alpha'], 
                    line_width=d['line_width'], line_dash=d['line_dash'])
        rd.append(r_line)
      
        rs.append(rd)
        
    p2.y_range = Range1d(ymax, ymin)  
    p2.yaxis.axis_label = 'pressure [mb]'
    
    p2.x_range = Range1d(xmin, xmax)
    p2.xaxis.axis_label = 'cooling rate [K/day]'
    
    items = [(d['label'], r) for r, d in zip(rs, pltdata)]
    legend = Legend(items=items, location=(10, -30))
    legend.label_text_font_size = '8pt'
    
    p2.add_layout(legend, 'right')  
    
    show(gridplot(p1, p2, ncols=2, plot_height=500))
    

def script_plt_cooling():
    pltdata = pltdata_cooling_1(
        ds_cool=DS_COOL,
        ds_cool_crd=DS_COOL_CRD,
        ds_cool_wgt=DS_COOL_WGT)
    plt_cooling_bokeh(pltdata=pltdata)
    print('''FIGURE. Cooling rate profiles.''')

In [9]:
def fmt_flux(ds_in, ilevels=None):
    ds = ds_in.copy(deep=True)
    
    for l in ('level', 'layer'):
        if l in ds.data_vars:
            if len(ds[l].dims) > 1:
                surface = {d: 0 for d in ds.dims if d != 'pressure'}
                coord_level = ds[l][surface]
                ds.coords[l] = ('pressure', coord_level)
            else:
                ds.coords[l] = ('pressure', ds[l])
    
    if ilevels:
        ds = ds.isel(pressure=ilevels)
        
    if 'igg' in ds.dims:
        ds = ds.sel(igg=[1])
        ds = ds.squeeze('igg').drop('igg')
    
    if 'g' in ds.dims:
        ds = ds.sum('g')
            
    if 'band' in ds.dims:
        try:
            ds = ds.squeeze('band').drop('band')
        except ValueError:
            ds = ds.sum('band')
                
    df = ds.to_dataframe()
    df = df.set_index('level', append=True)               
    return df


def tbdata_flux(ds_flux=None, ds_flux_crd=None, ds_flux_wgt=None):
    ilevels = [0, 23, -1]
    
    benchmark = {'label': 'CRD',
                 'df': fmt_flux(ds_flux_crd, ilevels=ilevels)}
    
    tbs = [{'label': 'WGT igg=10 wgt_flux=1',
            'df': fmt_flux(ds_flux_wgt, ilevels=ilevels)},
           {'label': 'CLIRAD',
            'df': fmt_flux(ds_flux, ilevels=ilevels)}]
    
    data = {'benchmark': benchmark, 'others': tbs}
    return data


def tb_flux(tbdata=None):
    benchmark = tbdata['benchmark']
    others = tbdata['others']
    
    def show_tb(d=None):
        display.display(d['df'])
        print('Table.', 'Fluxes.', d['label'])
        
    def show_tbdiff(d0=None, d=None):
        df = d['df']
        df.index = d0['df'].index
        display.display(df - d0['df'])
        print('Table.', 'Fluxes.', 
              '({}) - ({})'.format(d['label'], d0['label']))
        
    show_tb(benchmark)
    for d in others:
        show_tb(d)
        
    for d in others:
        show_tbdiff(d=d, d0=benchmark)
        
        
def script_tb_flux():
    data = tbdata_flux(ds_flux=DS_FLUX,
                       ds_flux_crd=DS_FLUX_CRD,
                       ds_flux_wgt=DS_FLUX_WGT)
    tb_flux(tbdata=data)

In [10]:
def band_map():
    '''
    Maps spectral bands in lblnew to spectral bands in clirad.
    '''
    lblnew2clirad = {'1': '1',
            '2': '2',
            '3a': '3',
            '3b': '4',
            '3c': '5',
            '4': '6', 
            '5': '7',
            '6': '8',
            '7': '9', 
            '8': '10',
            '9': '11'}
    
    clirad2lblnew = {clirad: lblnew 
                     for lblnew, clirad in lblnew2clirad.items()}
    return clirad2lblnew


def gasbands():
    bmap = band_map()
    gases = list(PARAM['molecule'].keys())
    
    bands = [bmap[str(b)] for b in PARAM['band']]
    return list(itertools.product(gases, bands))


def print_bestfit_params():
    print('Best-fit values for each (gas, band)')
    print('------------------------------------')
    for gas, band in gasbands():
        print('o {} band{}'.format(gas, band))
        param = bestfits.kdist_params(molecule=gas, band=band)
        for name, value in sorted(param.items()):
            print(2 * ' ' + '{} = {}'.format(name, value))

        
def print_input_param():        
    for name, value in PARAM.items():
        print('{} = {}'.format(name, value))
        
        
def print_lblnew_param():
    for name, value in sorted(PARAM_LBLNEW.items()):
        print('{} = {}'.format(name, value))


In [11]:
'''
Cooling rate profile by layer
'''
def script_coolr_bylayer():
    '''
    Displays cooling rate of each layer
    '''
    keys = ['CRD', 'CLIRAD `coolr_bands`']
    dss = [DS_COOL_CRD, DS_COOL]
    das = [fmt_cool(ds) for ds in dss]
    
    dfs = [da.to_dataframe() for da in das]
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'coolr_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))



In [12]:
'''
Fluxes by layer
'''
def script_flux_bylayer():
    '''
    Display fluxes of each layer
    '''
    keys = ['CRD', 'CLIRAD']
    dss = [DS_FLUX_CRD, DS_FLUX]
    dfs = [fmt_flux(ds) for ds in dss]
    
    df = pd.concat(dfs, axis=1, keys=keys)
    
    fname = 'fluxes_bylayer.csv'
    df.to_csv(fname)
    html_download = '''
    Download: <a href="./{fname}">{fname}</a>'''.format(fname=fname)
    
    with pd.option_context('display.max_rows', 999):
        display.display(df)
        
    display.display(display.HTML(html_download))


In [13]:
def show_html(s):
    display.display(display.HTML(s))

    
def show_markdown(s):
    display.display(display.Markdown(s))

    
def script():
    
    title = '### CLIRAD-LW'
    s_input = 'Input Parameters'
    s_lblnew_param = 'LBLNEW case compaired against'
    s_bestfits = 'Best-fit Parameters'
    s_plt_cooling = 'Cooling Rate Profiles'
    s_tb_flux = 'Flux Comparison'
    s_coolr_bylayer = 'Cooling Rate by Layer'
    s_flux_bylayer = 'Fluxes by Layer'
    
    # title
    show_markdown('### CLIRAD-LW')
    
    # TOC
    show_markdown('### Table of Contents')
    show_html(climahtml.getHTML_hrefanchor(s_input))
    show_html(climahtml.getHTML_hrefanchor(s_lblnew_param))
    show_html(climahtml.getHTML_hrefanchor(s_plt_cooling))
    show_html(climahtml.getHTML_hrefanchor(s_tb_flux))
    show_html(climahtml.getHTML_hrefanchor(s_bestfits))
    show_html(climahtml.getHTML_hrefanchor(s_coolr_bylayer))
    show_html(climahtml.getHTML_hrefanchor(s_flux_bylayer))
    
    # Input parameters
    show_html(climahtml.getHTML_idanchor(s_input))
    show_markdown(climahtml.getMarkdown_sectitle(s_input))
    print_input_param()
    
    # LBLNEW parameters 
    show_html(climahtml.getHTML_idanchor(s_lblnew_param))
    show_markdown(climahtml.getMarkdown_sectitle(s_lblnew_param))
    print_lblnew_param()
    
    # Cooling rate profiles
    show_html(climahtml.getHTML_idanchor(s_plt_cooling))
    show_markdown(climahtml.getMarkdown_sectitle(s_plt_cooling))
    script_plt_cooling()
    
    # Flux comparison
    show_html(climahtml.getHTML_idanchor(s_tb_flux))
    show_markdown(climahtml.getMarkdown_sectitle(s_tb_flux))
    script_tb_flux()
    
    # Best-fit values
    show_html(climahtml.getHTML_idanchor(s_bestfits))
    show_markdown(climahtml.getMarkdown_sectitle(s_bestfits))
    print_bestfit_params()
    
    # Cooling rate by layer
    show_html(climahtml.getHTML_idanchor(s_coolr_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_coolr_bylayer))
    script_coolr_bylayer()
    
    # Fluxes by layer
    show_html(climahtml.getHTML_idanchor(s_flux_bylayer))
    show_markdown(climahtml.getMarkdown_sectitle(s_flux_bylayer))
    script_flux_bylayer()
   
    

In [14]:
script()

### CLIRAD-LW

### Table of Contents

# Input Parameters

atmpro = mls
band = [3]
commitnumber = de5d43e
molecule = {'co2': 0.0004}
tsfc = 294


# LBLNEW case compaired against

...

Best-fit values for each (gas, band)
------------------------------------
o co2 band3a
  atmpro = mls
  band = 3a
  commitnumber = a06b618
  conc = 0.0004
  dv = 0.001
  klin = 6.375563e-24
  molecule = co2
  ng_adju = [0, 0, 0]
  ng_refs = [3, 2, 4]
  nv = 1000
  option_compute_btable = 0
  option_compute_ktable = 1
  option_wgt_flux = 1
  option_wgt_k = 1
  ref_pts = [(1, 250), (10, 250), (500, 250)]
  tsfc = 294
  vmax = 620
  vmin = 540
  w_diffuse = [(1.6, 1.6, 1.7), (1.75, 1.75), (1.55, 1.55, 1.6, 1.85)]
  wgt = [(0.7, 0.3, 0.7), (0.7, 0.6), (0.4, 0.5, 0.8, 0.95)]


# Cooling Rate by Layer

CRD           CLIRAD `coolr_bands`          
            layer    coolrg                layer    coolrg
pressure                                                  
0.000312      1.0  0.389489                  NaN       NaN
0.000312      NaN       NaN                  1.0  0.243179
0.000750      2.0  0.388176                  2.0  0.247010
0.001052      3.0  0.399332                  3.0  0.257655
0.001476      4.0  0.409658                  4.0  0.268771
0.002070      5.0  0.417947                  5.0  0.279476
0.002904      6.0  0.422956                  6.0  0.289445
0.004074      7.0  0.424201                  7.0  0.298274
0.005714      8.0  0.420278                  8.0  0.305312
0.008015      9.0  0.409925                  9.0  0.309766
0.011243     10.0  0.391847                 10.0  0.310405
0.015771     11.0  0.364466                 11.0  0.305660
0.022122     12.0  0.326236                 12.0  0.293116
0.031031     13.0  0.275434                 13.0  0.269958
0.043528     14.0  0.210205                 14.0  0.233203
0.061057     15.0  0.160402                 15.0  0.206052
0.085645     16.0  0.178620                 16.0  0.231262
0.120136     17.0  0.241901                 17.0  0.287418
0.168516     18.0  0.291570                 18.0  0.316873
0.236378     19.0  0.331875                 19.0  0.318927
0.331549     20.0  0.378121                 20.0  0.323591
0.465100     21.0  0.448609                 21.0  0.373628
0.652400     22.0  0.574815                 22.0  0.506149
0.915100     23.0  0.721690                 23.0  0.666639
1.283650     24.0  0.691988                 24.0  0.660908
1.800600     25.0  0.535288                 25.0  0.525999
2.525700     26.0  0.414367                 26.0  0.412404
3.542800     27.0  0.325918                 27.0  0.337497
4.969550     28.0  0.257415                 28.0  0.282809
6.970850     29.0  0.205556                 29.0  0.238383
9.778100     30.0  0.164672                 30.0  0.192288
13.715850    31.0  0.127783                 31.0  0.144560
19.239350    32.0  0.094104                 32.0  0.099522
26.987250    33.0  0.071226                 33.0  0.069202
37.855300    34.0  0.057832                 34.0  0.055466
53.100050    35.0  0.044322                 35.0  0.047749
73.887500    36.0  0.029910                 36.0  0.038228
97.662500    37.0  0.021253                 37.0  0.030106
121.437500   38.0  0.016832                 38.0  0.023781
145.212500   39.0  0.008260                 39.0  0.012418
168.987500   40.0 -0.001416                 40.0  0.000312
192.762500   41.0  0.000408                 41.0  0.000967
216.537500   42.0  0.013739                 42.0  0.014060
240.312500   43.0  0.025656                 43.0  0.025784
264.087500   44.0  0.035534                 44.0  0.034997
287.862500   45.0  0.044365                 45.0  0.042650
311.637500   46.0  0.052432                 46.0  0.049202
335.412500   47.0  0.059858                 47.0  0.055112
359.187500   48.0  0.066717                 48.0  0.060785
382.962500   49.0  0.073051                 49.0  0.066368
406.737500   50.0  0.078825                 50.0  0.071851
430.512500   51.0  0.083967                 51.0  0.077202
454.287500   52.0  0.088361                 52.0  0.082276
478.062500   53.0  0.091971                 53.0  0.086952
501.837500   54.0  0.094914                 54.0  0.091184
525.612500   55.0  0.097388                 55.0  0.095010
549.387500   56.0  0.099557                 56.0  0.098645
573.162500   57.0  0.101520                 57.0  0.102050
596.937500   58.0  0.103342                 58.0  0.105209
620.712500   59.0  0.105074                 59.0  0.108113
644.487500   60.0  0.106757                 60.0  0.110757
668.262500   61.0  0.108432                 61.0  0.113182
692.037500   62.0  0.110170                 62.0  0.115473
715.812500   63.0  0.112058                 63.0  0.117707
739.587500   64.0  0.114182                 64.0  0.120022
76

# Fluxes by Layer

CRD                           CLIRAD             \
                         flug       fldg      fnetg       flug       fldg   
pressure     level                                                          
0.000000e+00 1     -29.581934   0.000000 -29.581934        NaN        NaN   
1.000000e-08 1            NaN        NaN        NaN -29.702360   0.000002   
6.244000e-04 2     -29.581951   0.000046 -29.581905 -29.702376   0.000036   
8.759000e-04 3     -29.581958   0.000064 -29.581894 -29.702383   0.000050   
1.228600e-03 4     -29.581968   0.000091 -29.581877 -29.702392   0.000070   
1.723400e-03 5     -29.581981   0.000128 -29.581853 -29.702405   0.000099   
2.417400e-03 6     -29.581999   0.000180 -29.581819 -29.702423   0.000140   
3.390900e-03 7     -29.582024   0.000254 -29.581770 -29.702447   0.000198   
4.756500e-03 8     -29.582059   0.000358 -29.581701 -29.702482   0.000281   
6.672000e-03 9     -29.582108   0.000502 -29.581606 -29.702530   0.000398   
9.358900e-03 10    -29.582175   0.000700 -29.581476 -29.702597   0.000564   
1.312780e-02 11    -29.582270   0.000970 -29.581301 -29.702692   0.000797   
1.841450e-02 12    -29.582404   0.001332 -29.581072 -29.702825   0.001122   
2.583020e-02 13    -29.582595   0.001810 -29.580786 -29.703016   0.001570   
3.623230e-02 14    -29.582874   0.002428 -29.580446 -29.703292   0.002178   
5.082340e-02 15    -29.583294   0.003211 -29.580083 -29.703699   0.002989   
7.129060e-02 16    -29.583932   0.004238 -29.579694 -29.704300   0.004090   
1.000000e-01 17    -29.584849   0.005762 -29.579087 -29.705143   0.005719   
1.402710e-01 18    -29.586078   0.008145 -29.577933 -29.706248   0.008195   
1.967600e-01 19    -29.587673   0.011691 -29.575982 -29.707645   0.011713   
2.759970e-01 20    -29.589660   0.016793 -29.572867 -29.709332   0.016393   
3.871000e-01 21    -29.591970   0.024080 -29.567890 -29.711213   0.022534   
5.431000e-01 22    -29.594255   0.034654 -29.559600 -29.712953   0.031179   
7.617000e-01 23    -29.595105   0.050390 -29.544716 -29.713264   0.044598   
1.068500e+00 24    -29.591321   0.072834 -29.518487 -29.709350   0.064914   
1.498800e+00 25    -29.582270   0.099055 -29.483215 -29.700720   0.089975   
2.102400e+00 26    -29.570761   0.125820 -29.444942 -29.689615   0.116484   
2.949000e+00 27    -29.557826   0.154439 -29.403387 -29.676824   0.145055   
4.136600e+00 28    -29.543780   0.186244 -29.357537 -29.662409   0.178124   
5.802500e+00 29    -29.528985   0.222247 -29.306739 -29.646508   0.218038   
8.139200e+00 30    -29.513547   0.263706 -29.249841 -29.628883   0.266404   
1.141700e+01 31    -29.497521   0.311618 -29.185903 -29.609463   0.321652   
1.601470e+01 32    -29.482221   0.365913 -29.116308 -29.589524   0.380454   
2.246400e+01 33    -29.471270   0.426855 -29.044416 -29.573132   0.440100   
3.151050e+01 34    -29.467531   0.499443 -28.968088 -29.563628   0.504763   
4.420010e+01 35    -29.472291   0.591134 -28.881157 -29.562278   0.586796   
6.200000e+01 36    -29.493049   0.705345 -28.787704 -29.577279   0.702487   
8.577500e+01 37    -29.545127   0.841658 -28.703468 -29.626101   0.858983   
1.095500e+02 38    -29.617933   0.974322 -28.643612 -29.699220   1.016898   
1.333250e+02 39    -29.706814   1.110606 -28.596207 -29.791231   1.175890   
1.571000e+02 40    -29.818628   1.245683 -28.572946 -29.908647   1.328282   
1.808750e+02 41    -29.964343   1.387409 -28.576934 -30.062494   1.483008   
2.046500e+02 42    -30.139695   1.563911 -28.575784 -30.245885   1.669122   
2.284250e+02 43    -30.329751   1.792660 -28.537091 -30.443813   1.906652   
2.522000e+02 44    -30.529194   2.064357 -28.464836 -30.650300   2.185762   
2.759750e+02 45    -30.736253   2.371491 -28.364762 -30.863391   2.497425   
2.997500e+02 46    -30.949554   2.709740 -28.239815 -31.081621   2.835784   
3.235250e+02 47    -31.167843   3.075694 -28.092149 -31.303694   3.196442   
3.473000e+02 48    -31.389935   3.466366 -27.923569 -31.528162   3.576139   
3.710750e+02 49    -31

In [15]:
display.HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')